# Classical Regression Attempt to Fit Dataset

In [ ]:
import numpy as np

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import os
# Prepare the 




In [ ]:
# Load the 4-parameter ML dataset with Kbearing at the C-vertex
four_params_Kbearing_c = np.load('data/major_params_Kbearing_c.npy')

# Define feature and target columns
feature_cols = [0, 1, 2, 3]  # height_factor, thickness, w_over_r, a_over_c
target_cols = [4]  # Kbearing_c

## Classical Regression Techniques for Multi-Parameter Modeling

### Technical Background

**1. Multiple Linear Regression (MLR)**
- Form: $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \epsilon$
- Assumes linear relationships between features and target
- Uses ordinary least squares (OLS) to minimize residuals

**2. Polynomial Regression**
- Adds polynomial terms: $y = \beta_0 + \beta_1 x_1 + \beta_2 x_1^2 + \beta_3 x_2 + \beta_4 x_2^2 + ...$
- Captures non-linear relationships
- Can include interaction terms: $\beta_{ij} x_i x_j$

**3. Power Law Models**
- Form: $y = A \cdot x_1^{\alpha_1} \cdot x_2^{\alpha_2} \cdot x_3^{\alpha_3} \cdot x_4^{\alpha_4}$
- Common in engineering/physics applications
- Linearized via log transformation: $\log y = \log A + \alpha_1 \log x_1 + \alpha_2 \log x_2 + ...$

**4. Exponential Models**
- Form: $y = A e^{\beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4}$
- Useful for growth/decay phenomena
- Linearized: $\log y = \log A + \beta_1 x_1 + \beta_2 x_2 + ...$

**5. Mixed Models with Interaction Terms**
- Combines linear, polynomial, and interaction effects
- Example: $y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 + \beta_4 x_1^2 + ...$

### Implementation Strategy
1. Start with simple linear regression
2. Add polynomial terms systematically  
3. Test interaction terms between parameters
4. Use statistical tests (R², F-test, p-values) to evaluate model quality
5. Apply regularization (Ridge/Lasso) if needed for feature selection

In [ ]:

# Extract features and targets
X = four_params_Kbearing_c[:, feature_cols]  # [height_factor, thickness, w_over_r, a_over_c]
y = four_params_Kbearing_c[:, target_cols].ravel()  # Kbearing_c

# Create feature names for interpretability
feature_names = ['height_factor', 'thickness', 'w_over_r', 'a_over_c']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset shape: {X.shape}")
print(f"Feature names: {feature_names}")
print(f"Target range: [{y.min():.2f}, {y.max():.2f}]")

In [ ]:
# 1. Multiple Linear Regression (Baseline)
print("=== 1. Multiple Linear Regression ===")
mlr = LinearRegression()
mlr.fit(X_train, y_train)

# Predictions
y_pred_mlr = mlr.predict(X_test)
r2_mlr = r2_score(y_test, y_pred_mlr)
mse_mlr = mean_squared_error(y_test, y_pred_mlr)

print(f"R² Score: {r2_mlr:.4f}")
print(f"MSE: {mse_mlr:.4f}")
print(f"RMSE: {np.sqrt(mse_mlr):.4f}")

# Display the linear equation
print("\nLinear Model Equation:")
print(f"Kbearing_c = {mlr.intercept_:.4f}")
for i, (name, coef) in enumerate(zip(feature_names, mlr.coef_)):
    print(f"           + {coef:.4f} * {name}")

# Statistical significance testing
def calculate_p_values(model, X, y):
    """Calculate p-values for regression coefficients"""
    n = X.shape[0]
    k = X.shape[1]
    
    # Predictions and residuals
    y_pred = model.predict(X)
    residuals = y - y_pred
    mse = np.sum(residuals**2) / (n - k - 1)
    
    # Covariance matrix
    X_with_intercept = np.column_stack([np.ones(n), X])
    cov_matrix = mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept)
    
    # Standard errors and t-statistics
    std_errors = np.sqrt(np.diag(cov_matrix))
    coefficients = np.concatenate([[model.intercept_], model.coef_])
    t_stats = coefficients / std_errors
    
    # P-values (two-tailed test)
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), n - k - 1))
    
    return p_values, std_errors

p_values, std_errors = calculate_p_values(mlr, X_train, y_train)
print("\nStatistical Significance:")
print(f"Intercept: p-value = {p_values[0]:.4f}")
for i, (name, p_val) in enumerate(zip(feature_names, p_values[1:])):
    significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
    print(f"{name}: p-value = {p_val:.4f} {significance}")

In [ ]:
# 2. Polynomial Regression with Interaction Terms
print("\n=== 2. Polynomial Regression with Interactions ===")

# Create polynomial features (degree=2 includes interactions)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Fit polynomial regression
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)

# Predictions
y_pred_poly = poly_reg.predict(X_test_poly)
r2_poly = r2_score(y_test, y_pred_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)

print(f"R² Score: {r2_poly:.4f}")
print(f"MSE: {mse_poly:.4f}")
print(f"RMSE: {np.sqrt(mse_poly):.4f}")

# Display feature names and coefficients
feature_names_poly = poly_features.get_feature_names_out(feature_names)
print(f"\nPolynomial Model ({len(feature_names_poly)} terms):")
print(f"Kbearing_c = {poly_reg.intercept_:.4f}")
for name, coef in zip(feature_names_poly, poly_reg.coef_):
    if abs(coef) > 1e-6:  # Only show significant coefficients
        print(f"           + {coef:.6f} * {name}")

print(f"\nImprovement over linear: {r2_poly - r2_mlr:.4f} R² points")

In [ ]:
# 2b. Extended Polynomial Regression with Higher Orders and Inverse Terms
print("\n=== 2b. Extended Polynomial with Higher Orders and Inverse Terms ===")

# Create extended feature matrix with higher order and inverse terms
def create_extended_polynomial_features(X, max_degree=3, min_degree=-3):
    """
    Create extended polynomial features including inverse terms
    X: input features
    max_degree: maximum polynomial degree (positive)
    min_degree: minimum polynomial degree (negative for inverse terms)
    """
    n_samples, n_features = X.shape
    extended_features = []
    feature_names_extended = []
    
    # Add original features (degree 1)
    extended_features.append(X)
    feature_names_extended.extend([f"{name}" for name in feature_names])
    
    # Add positive polynomial terms (degree 2 to max_degree)
    for degree in range(2, max_degree + 1):
        for i in range(n_features):
            extended_features.append((X[:, i] ** degree).reshape(-1, 1))
            feature_names_extended.append(f"{feature_names[i]}^{degree}")
    
    # Add inverse terms (degree -1 to min_degree)
    for degree in range(-1, min_degree - 1, -1):
        for i in range(n_features):
            # Avoid division by zero - add small epsilon if needed
            safe_feature = X[:, i] + 1e-10 * (X[:, i] == 0)
            extended_features.append((safe_feature ** degree).reshape(-1, 1))
            feature_names_extended.append(f"{feature_names[i]}^{degree}")
    
    # Add interaction terms (x_i * x_j)
    for i in range(n_features):
        for j in range(i + 1, n_features):
            interaction = (X[:, i] * X[:, j]).reshape(-1, 1)
            extended_features.append(interaction)
            feature_names_extended.append(f"{feature_names[i]} * {feature_names[j]}")
    
    # Add mixed polynomial-inverse interactions (x_i^2 * x_j^-1, etc.)
    for i in range(n_features):
        for j in range(n_features):
            if i != j:
                # x_i^2 * x_j^-1
                safe_feature_j = X[:, j] + 1e-10 * (X[:, j] == 0)
                mixed_term = (X[:, i]**2 * safe_feature_j**(-1)).reshape(-1, 1)
                extended_features.append(mixed_term)
                feature_names_extended.append(f"{feature_names[i]}^2 * {feature_names[j]}^-1")
    
    return np.hstack(extended_features), feature_names_extended

# Create extended polynomial features for training and test sets
X_train_extended, feature_names_extended = create_extended_polynomial_features(X_train, max_degree=3, min_degree=-3)
X_test_extended, _ = create_extended_polynomial_features(X_test, max_degree=3, min_degree=-3)

print(f"Extended feature matrix shape: {X_train_extended.shape}")
print(f"Number of features: {len(feature_names_extended)}")

# Check for infinite or NaN values
if np.any(np.isinf(X_train_extended)) or np.any(np.isnan(X_train_extended)):
    print("WARNING: Extended features contain infinite or NaN values!")
    # Replace inf and nan with large/small finite values
    X_train_extended = np.nan_to_num(X_train_extended, nan=0.0, posinf=1e10, neginf=-1e10)
    X_test_extended = np.nan_to_num(X_test_extended, nan=0.0, posinf=1e10, neginf=-1e10)

# Fit extended polynomial regression
extended_poly_reg = LinearRegression()
extended_poly_reg.fit(X_train_extended, y_train)

# Predictions
y_pred_extended = extended_poly_reg.predict(X_test_extended)
r2_extended = r2_score(y_test, y_pred_extended)
mse_extended = mean_squared_error(y_test, y_pred_extended)

print(f"R² Score: {r2_extended:.4f}")
print(f"MSE: {mse_extended:.4f}")
print(f"RMSE: {np.sqrt(mse_extended):.4f}")

# Show most important features (largest absolute coefficients)
coef_importance = np.abs(extended_poly_reg.coef_)
top_indices = np.argsort(coef_importance)[::-1][:10]  # Top 10 features

print(f"\nTop 10 Most Important Features:")
print(f"Kbearing_c = {extended_poly_reg.intercept_:.6f}")
for idx in top_indices:
    if coef_importance[idx] > 1e-10:  # Only show significant coefficients
        print(f"           + {extended_poly_reg.coef_[idx]:>12.6f} * {feature_names_extended[idx]}")

print(f"\nImprovement over basic polynomial: {r2_extended - r2_poly:.4f} R² points")

# Store results for comparison
models_extended = {
    'Linear': (r2_mlr, mse_mlr, y_pred_mlr),
    'Polynomial': (r2_poly, mse_poly, y_pred_poly),
    'Extended Polynomial': (r2_extended, mse_extended, y_pred_extended)
}

In [ ]:
# 3. Power Law Model (Log-Linear Transformation)
print("\n=== 3. Power Law Model ===")

# Check for positive values (required for log transformation)
if np.all(X > 0) and np.all(y > 0):
    # Log transformation for power law: log(y) = log(A) + α₁log(x₁) + α₂log(x₂) + ...
    X_log = np.log(X_train)
    y_log = np.log(y_train)
    
    # Fit log-linear model
    power_reg = LinearRegression()
    power_reg.fit(X_log, y_log)
    
    # Predictions (transform back from log space)
    X_test_log = np.log(X_test)
    y_pred_log = power_reg.predict(X_test_log)
    y_pred_power = np.exp(y_pred_log)
    
    r2_power = r2_score(y_test, y_pred_power)
    mse_power = mean_squared_error(y_test, y_pred_power)
    
    print(f"R² Score: {r2_power:.4f}")
    print(f"MSE: {mse_power:.4f}")
    print(f"RMSE: {np.sqrt(mse_power):.4f}")
    
    # Display power law equation
    A = np.exp(power_reg.intercept_)
    print(f"\nPower Law Model:")
    print(f"Kbearing_c = {A:.4f}", end="")
    for i, (name, alpha) in enumerate(zip(feature_names, power_reg.coef_)):
        print(f" * {name}^{alpha:.4f}", end="")
    print()
    
    print(f"\nImprovement over linear: {r2_power - r2_mlr:.4f} R² points")
else:
    print("Cannot apply power law model: data contains non-positive values")
    y_pred_power = None
    r2_power = None

In [ ]:
# 4. Regularized Polynomial Regression (Feature Selection)
print("\n=== 4. Regularized Polynomial Regression ===")

# Use Ridge regression to handle overfitting in polynomial model
ridge_poly = Ridge(alpha=1.0)
ridge_poly.fit(X_train_poly, y_train)

y_pred_ridge = ridge_poly.predict(X_test_poly)
r2_ridge = r2_score(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

print(f"Ridge Regression (α=1.0):")
print(f"R² Score: {r2_ridge:.4f}")
print(f"MSE: {mse_ridge:.4f}")
print(f"RMSE: {np.sqrt(mse_ridge):.4f}")

# Use Lasso for feature selection
lasso_poly = Lasso(alpha=0.1)
lasso_poly.fit(X_train_poly, y_train)

y_pred_lasso = lasso_poly.predict(X_test_poly)
r2_lasso = r2_score(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

print(f"\nLasso Regression (α=0.1):")
print(f"R² Score: {r2_lasso:.4f}")
print(f"MSE: {mse_lasso:.4f}")
print(f"RMSE: {np.sqrt(mse_lasso):.4f}")

# Show selected features (non-zero coefficients)
selected_features = feature_names_poly[lasso_poly.coef_ != 0]
selected_coefs = lasso_poly.coef_[lasso_poly.coef_ != 0]

print(f"\nLasso Selected Features ({len(selected_features)} out of {len(feature_names_poly)}):")
print(f"Kbearing_c = {lasso_poly.intercept_:.4f}")
for name, coef in zip(selected_features, selected_coefs):
    print(f"           + {coef:.6f} * {name}")

In [ ]:
# 5. Model Comparison and Visualization
print("\n=== Model Comparison Summary ===")

models = {
    'Linear': (r2_mlr, mse_mlr, y_pred_mlr),
    'Polynomial': (r2_poly, mse_poly, y_pred_poly),
    'Ridge': (r2_ridge, mse_ridge, y_pred_ridge),
    'Lasso': (r2_lasso, mse_lasso, y_pred_lasso)
}

if r2_power is not None:
    models['Power Law'] = (r2_power, mse_power, y_pred_power)

# Create comparison table
comparison_df = pd.DataFrame({
    'Model': list(models.keys()),
    'R² Score': [models[name][0] for name in models.keys()],
    'MSE': [models[name][1] for name in models.keys()],
    'RMSE': [np.sqrt(models[name][1]) for name in models.keys()]
})

comparison_df = comparison_df.sort_values('R² Score', ascending=False)
print(comparison_df.to_string(index=False, float_format='%.4f'))

# Identify best model
best_model_name = comparison_df.iloc[0]['Model']
best_r2 = comparison_df.iloc[0]['R² Score']
print(f"\nBest Model: {best_model_name} (R² = {best_r2:.4f})")

In [ ]:
# 6. Visualization: Predicted vs Actual
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

plot_idx = 0
for name, (r2, mse, y_pred) in models.items():
    if plot_idx < len(axes):
        ax = axes[plot_idx]
        
        # Predicted vs Actual scatter plot
        ax.scatter(y_test, y_pred, alpha=0.6, s=20)
        
        # Perfect prediction line (y=x)
        min_val = min(y_test.min(), y_pred.min())
        max_val = max(y_test.max(), y_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
        
        ax.set_xlabel('Actual Kbearing_c')
        ax.set_ylabel('Predicted Kbearing_c')
        ax.set_title(f'{name} Model\\nR² = {r2:.4f}, RMSE = {np.sqrt(mse):.4f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plot_idx += 1

# Remove empty subplots
for idx in range(plot_idx, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

# Feature importance plot for the best polynomial model
if best_model_name == 'Polynomial':
    plt.figure(figsize=(12, 6))
    coef_importance = np.abs(poly_reg.coef_)
    sorted_idx = np.argsort(coef_importance)[::-1]
    
    plt.bar(range(len(coef_importance)), coef_importance[sorted_idx])
    plt.xticks(range(len(coef_importance)), [feature_names_poly[i] for i in sorted_idx], rotation=45, ha='right')
    plt.ylabel('|Coefficient|')
    plt.title('Feature Importance (Polynomial Model)')
    plt.tight_layout()
    plt.show()